# RecpackFusion

In [0]:
!pip install wandb einops &> /dev/null
!rm -r recpackfusion
!git clone https://github.com/gabriben/recfusion.git
!cd recfusion; pip install . &> /dev/null

In [0]:
from recpack.quick import quick_train


model = 'RecFusionMLP'
dataset = 'MovieLens1M'
prep_hypers = {
    'ds_path': '/dbfs/datasets/RecSys/',  
    'min_rating': 4,
    'min_items_per_user': 5,
    'min_users_per_item': 5,
    'generalization': 'StrongGeneralization', # WeakGeneralization,
    "train_val_test": [0.8, 0.1, 0.1],
    'force_even_items': False
}
train_hypers = {
    # "max_epochs":1,
    "stop_early": False, # "max_iter_no_change" : 10, "min_improvement": 0.001,
    # "validation_sample_size": 1000,
    # "dim_bottleneck_layer" : 128,
    'T': 2, 
    # 'p_dnns_depth': 5,
    # "T": 100,
    # "batch_size" : 200,
    # "M" : 200,
    "p_dnns_depth": 2,
    # "anneal_steps" : 20    
    # "time_embedding_as_input" : True  
    #"schedule_type": 'fixed',
    #'jascha_bin_process': True,
    #'b_start' : 0.1,    
}
val_metric = {'NDCGK':100} #{'NDCGK':100}    
test_metrics = {
    'NDCGK' : [10, 20, 50, 100],
    'RecallK' : [10, 20, 50],
    'HitK': [20, 50, 100],
    'CalibratedRecallK': [10, 20, 50]
} 

In [0]:
neural_baselines = ["RecVAE", "CODIGEM", "MultVAE"] 
recfusion = ["RecFusionBin", "RecFusionMLPVar", "RecFusionMLPT", "RecFusionMLP"]
recfusion_unet = ["RecFusionUnet1D", "RecFusionUnet2D"]
basic_baselines = ["Random", "Popularity", "EASE", "SLIM"]

In [0]:
import pandas as pd
from pyspark.sql.functions import col, udf
from pyspark.sql.types import LongType
import os

# Declare the function and create the UDF
@udf("float")
def training_func(model):

  os.environ["WANDB_API_KEY"] = ""
  os.environ["WANDB_PROJECT"] = ""

  m, v = quick_train(model, dataset, prep_hypers, train_hypers, val_metric, test_metrics)
  loss = v
  return v

# The df contains on each row a combination of the hyper parameters
df = spark.createDataFrame(pd.DataFrame({
    "model": ["RecFusionMLP"] * 10
    # "model": # basic_baselines
}))

df.repartition(df.count()).withColumn("loss", training_func(col("model"))).show()